When operating in individual elements, we observe big slowdowns in Modin.
TODO: The code below crosses rows in every iteration. Create another version that deals with one row at a time.

In [1]:
import utils
%include preamble.py

2022-12-24 07:15:47,558	INFO worker.py:1528 -- Started a local Ray instance.


In [2]:
modin_df, pandas_df = utils.get_two_dfs("../../datasets/yellow_tripdata_2015-01.csv")

In [3]:
%%run_twice DF_PH, modin_df, pandas_df

# Taken from: erikbruin/nlp-on-student-writing-eda

DF_PH['discourse_nr'] = 1
counter = 1

# I had to reduce the number of iterations to 15 otherwise, it takes _A LOT_ of time for
# Modin.
for i in range(1, 15):
  if DF_PH.loc[i, 'VendorID'] == DF_PH.loc[i-1, 'VendorID']:
    counter += 1
    DF_PH.loc[i, 'discourse_nr'] = counter
  else:
    counter = 1
    DF_PH.loc[i, 'discourse_nr'] = counter

In [4]:
modin_time = _TIMED_CELL1
pandas_time = _TIMED_CELL2
print(f"Modin time: {modin_time:.1f}s")
print(f"Pandas time: {pandas_time:.1f}s")
slowdown = modin_time / pandas_time
utils.print_md(f"### Modin is {slowdown:.1f}x slower.")

Modin time: 48.3s
Pandas time: 0.0s


### Modin is 1914.5x slower.

In [5]:
assert (modin_df['discourse_nr'] == pandas_df['discourse_nr']).all()